In [52]:
# Calcula el pago de luz para tarifas residenciales.

import pandas as pd
import numpy as np

In [53]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

# Escenarios.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/data" for x in dirs]
csv = ".csv"
path_cfe = path + "CFE/"
f_tarifa = "Tarifa_1.csv"
f_dac = "DAC.csv"
f_regiones = "Regiones.csv"

tarifa = pd.read_csv(path_cfe + f_tarifa)
dac = pd.read_csv(path_cfe + f_dac)
regiones = pd.read_csv(path_cfe + f_regiones)

# Tipos de escalones para las tarifas.
escalon_0 = tarifa["Escalon"].unique()

# Se establece la tarifa DAC para cada estado.
regiones[ ["DAC", "Equilibrio"] ] = np.nan
for i in regiones.itertuples():
    regiones.loc[ i[0], ["DAC", "Equilibrio"] ] = ( 
        dac.drop(8).loc[ dac["Region"] == i.Region,
        ["DAC", "Equilibrio"] ].values[0] )


In [139]:
k = 0

cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
    "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

# Información mensual.
df_0 = pd.read_csv(path_data[k] + csv)
# Información anual.
df_y = pd.read_csv(path_data[k] + "_yearly" + csv)

# Promediamos el consumo y usuarios anuales.
if k == 0:
    df = df_y.groupby("CVE_INEGI").mean()
    df["Tarifa"] = ""

    # Recuperamos la información de las tarifas.
    for i in df.itertuples():
        df.loc[i[0], "Tarifa"] = df_y.query(
            "CVE_INEGI==" + str(i[0]) +
            " & Año==2010" )["Tarifa"].values[0]

# Redondeamos la cantidad de usuarios.
df[["Usuarios_1*", "Usuarios_DAC"]] = df[
    ["Usuarios_1*", "Usuarios_DAC"]].round()

# Retiramos la columna de año, que ya no tiene sentido.
df.drop(columns = "Año", inplace = True)

# Sumamos el consumo correspondiente al verano.
df_2 = df_0.where(df_0["Verano"] == True)
df_2 = df_2[df_2["CVE_INEGI"].notna()]
df_2 = df_2.groupby(["CVE_INEGI", "Mes"]
    ).mean().reset_index()
df_2 = df_2.groupby(["CVE_INEGI"]
    ).sum()#.reset_index()
df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]

# Sumamos el consumo correspondiente al invierno.
df_2 = df_0.where(df_0["Verano"] == False)
df_2 = df_2[df_2["CVE_INEGI"].notna()]
df_2 = df_2.groupby(["CVE_INEGI", "Mes"]
    ).mean().reset_index()
df_2 = df_2.groupby(["CVE_INEGI"]
    ).sum()#.reset_index()
df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]

# Retiramos las columnas de consumo total anual.
df.drop(columns = ["Consumo_1*", "Consumo_DAC"], inplace = True)

# Unimos las tarifas 1 y DAC.
df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
df["Usuarios_1*"] += df["Usuarios_DAC"]
df.drop("Usuarios_DAC", axis = 1, inplace = True)

df.reset_index(inplace = True)

# Se crean columnas para los
# escalones de verano e invierno.
verano_escalon = []
invierno_escalon = []
for e in escalon_0:
    verano_escalon.append("Verano_" + e)
    invierno_escalon.append("Invierno_" + e)
escalon = [verano_escalon,
    invierno_escalon]
df[verano_escalon] = np.nan
df[invierno_escalon] = np.nan

# Se establece el valor de consumo límite para cada escalon.
for i in df.itertuples():
    for p in range(len(escalon_0)):
        df.loc[i[0], verano_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' + escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 6 )
        df.loc[i[0], invierno_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' +  escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 6 )

# Columnas para consumo en verano e invierno.
estacion = ["Verano_", "Invierno_"]
verano_escalon_cons = [ "Cons_" +
    x for x in verano_escalon ]
invierno_escalon_cons = [ "Cons_" +
    x for x in invierno_escalon ]
escalon_cons = [ verano_escalon_cons,
    invierno_escalon_cons ]

# Columnas para verano.
df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[0]] = df[escalon[0]].copy()
#df[estacion[0] + "Cons_DAC"] = np.nan
df[estacion[0] + "Pago"] = 0

# Columnas para invierno.
df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[1]] = df[escalon[1]].copy()
#df[estacion[1] + "Cons_DAC"] = np.nan
df[estacion[1] + "Pago"] = 0

df["DAC_Pago"] = 0

df

,CVE_INEGI,CVE_ENT,CVE_MUN,Usuarios_1*,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Tarifa,Verano_Consumo_1*,Invierno_Consumo_1*,Verano_Basico,Verano_Intermedio_bajo,Verano_Intermedio_alto,Verano_Excedente,Invierno_Basico,Invierno_Intermedio_bajo,Invierno_Intermedio_alto,Invierno_Excedente,Verano_Consumo_Usuario,Cons_Verano_Basico,Cons_Verano_Intermedio_bajo,Cons_Verano_Intermedio_alto,Cons_Verano_Excedente,Verano_Pago,Invierno_Consumo_Usuario,Cons_Invierno_Basico,Cons_Invierno_Intermedio_bajo,Cons_Invierno_Intermedio_alto,Cons_Invierno_Excedente,Invierno_Pago,DAC_Pago
0,1001,1.0,1.0,254974.0,-102.295872,21.811400,27.182119,13.074257,20.159075,522.051880,-482.033417,507.881513,-8.212147,1.347620,442.348606,845118.000000,1.374286e+11,23.691382,4.142857,7.245778,162160.723559,NaN,11.211926,116635.924759,1,1.920337e+08,1.693105e+08,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,753.149993,450.0,390.0,0.0,1500.0,0,664.030341,450.0,390.0,0.0,1500.0,0,0
1,1002,1.0,2.0,11557.0,-102.045590,22.126479,26.460970,12.328507,19.395846,429.399714,-494.662554,556.378653,-12.173154,0.947158,358.376980,46075.142857,3.491166e+09,23.034981,4.142857,0.847945,75705.155679,NaN,11.211926,54337.426603,1,7.759527e+06,6.846582e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,671.413599,450.0,390.0,0.0,1500.0,0,592.418585,450.0,390.0,0.0,1500.0,0,0
2,1003,1.0,3.0,16736.0,-102.704911,21.900640,27.009948,12.518081,19.773620,621.113330,-448.972848,462.771497,-4.712476,0.947242,519.512183,55283.142857,4.358511e+09,23.082358,4.142857,0.598719,78733.145869,NaN,11.211926,92335.783229,1,1.122328e+07,9.884636e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,670.606872,450.0,390.0,0.0,1500.0,0,590.621186,450.0,390.0,0.0,1500.0,0,0
3,1004,1.0,4.0,3788.0,-102.297038,22.360619,26.527926,12.449918,19.528949,448.636825,-512.037785,557.611886,-13.815264,0.775050,377.125221,15363.000000,1.449504e+09,23.041425,4.142857,1.196649,94221.940180,NaN,11.211926,12838.354454,1,2.510776e+06,2.213835e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,662.823763,450.0,390.0,0.0,1500.0,0,584.433834,450.0,390.0,0.0,1500.0,0,0
4,1005,1.0,5.0,30597.0,-102.445700,21.932112,26.925495,12.886637,19.890029,531.063800,-460.285867,479.792441,-7.156598,1.232661,442.897088,112079.000000,1.318374e+10,23.204577,4.142857,2.242050,117305.832917,NaN,11.211926,49989.509158,1,2.141233e+07,1.878628e+07,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,699.817794,450.0,390.0,0.0,1500.0,0,613.990777,450.0,390.0,0.0,1500.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,3923.0,-101.714324,22.407345,26.185634,12.639281,19.411589,357.411519,-506.235879,590.748675,-22.991064,1.840217,288.391072,18889.000000,1.104084e+09,23.055471,4.000000,0.508554,58432.484498,NaN,11.268627,37142.598662,1,2.097387e+06,1.593920e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,534.638540,450.0,390.0,0.0,1500.0,0,406.301266,450.0,390.0,0.0,1500.0,0,0
2465,32055,32.0,55.0,8847.0,-102.851571,22.298130,26.407706,12.344591,19.528930,558.532327,-469.145342,489.120322,-9.259408,0.741970,468.364202,29902.000000,2.396876e+09,22.681947,4.142857,0.138660,80134.667679,NaN,11.268627,215650.560221,1,5.127218e+06,3.901293e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,579.543129,450.0,390.0,0.0,1500.0,0,440.973591,450.0,390.0,0.0,1500.0,0,0
2466,32056,32.0,56.0,37702.0,-102.678055,22.735730,25.725354,11.868202,18.824115,439.723002,-530.293745,536.387068,-19.077788,1.257958,374.477437,142958.571429,2.484715e+10,22.429208,4.142857,3.272704,173750.774044,NaN,11.268627,43682.101050,1,2.610999e+07,1.966034e+07,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,692.535907,450.0,390.0,0.0,1500.0,0,521.466643,450.0,390.0,0.0,1500.0,0,0
2467,32057,32.0,57.0,15674.0,-102.312009,22.755625,2

In [140]:
IVA = 1.16
FIJO_DAC = 121.75

# Iteramos para verano e invierno.
for e in [0, 1]:

    # Offset de columnas de verano e invierno.
    if e == 0: s = 0
    else: s = 6

    col = np.concatenate( [
        [estacion[e] + "Consumo_Usuario"],
        escalon_cons[e]
        ] )

    # Calculamos los escalones de consumo
    # de acuerdo con cada límite.
    for i in range(len(col[:-1])):
        df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

    # Eliminamos valores negativos.
    df[col[1:]] = df[col[1:]].clip( lower = 0 )

    col = np.concatenate( [ escalon_cons[e] ] )
    col = np.flip(col)

    # Recorremos los escalones a su posición adecuada.
    for i in range(len(col[:-1])):
        df[col[i]] = df[col[i + 1]]

    for i in df.itertuples():
            
        # No se está en tarifa DAC.
        if not( ( i.Verano_Consumo_Usuario +
            i.Invierno_Consumo_Usuario ) > i.Verano_Excedente):
            # Si el consumo es menor al límite básico, el
            # escalon básico es igual al consumo total.
            if i[37 + s] == 0:
                df.loc[i[0], escalon_cons[e][0]] = i[36 + s]
            # Se elimina  el escalón intermedio alto
            # para las tarifas donde no existe.
            # Si el precio de este escalon es cero,
            # este paso no debería ser necesario.
            if i.Tarifa in ["1", "1A", "1B"]:
                df.loc[i[0], escalon_cons[e][2]] = 0
            # Si el consumo excede un escalón, 
            # el consumo de un escalón es su límite.
            for j in range(3):
                if i[37 + s + j + 1] > 0:
                    df.loc[i[0], escalon_cons[e][j]] = i[
                        28 + s * 4 // 6 + j]
        
        # Sí está en tarifa DAC.
        else:
            # El resto de pagos es igual a cero.
            df.loc[i[0], col] = 0

            # Se calcula el pago con todo el consumo general.
            df.loc[i[0], "DAC_Pago"] = ( (
                i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                * regiones.loc[
                regiones["CVE_ENT"] == i.CVE_ENT,
                "DAC"].values[0] + FIJO_DAC )

    # Se calcula el pago por temporada.
    df[estacion[e] + "Pago"] = ( df[
        np.concatenate( [ escalon_cons[e]
        ] ) ].sum(axis = 1) * IVA )

# Se eliminan las columnas de apoyo.
df.drop( columns = np.concatenate( [
    escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
    ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
    ] ), inplace = True )

# Se calcula el pago total.
df["Pago_real"] = ( df[[estacion[0] + "Pago",
    estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

# Se eliminan los pagos totales.
df.drop( columns = [ estacion[0] + "Pago",
    estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

# Unimos el consumo de verano e invierno.
df["Consumo_1*"] = df["Verano_Consumo_1*"] + df["Invierno_Consumo_1*"]
df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
df["Pago_sin_subsidio"] = np.nan

# Calculamos el precio sin subsidio, es decir,
# con un precio plano igual al precio marginal.
for i in df.itertuples():
    df.loc[i[0], "Pago_sin_subsidio"] = ( 
        i.Consumo_Usuario * regiones.loc[
        regiones["CVE_ENT"] == i.CVE_ENT,
        "Equilibrio" ].values[0] * IVA )

# Se calcula el subsidio.
df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"] -
    df["Pago_real"] )
df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

df.to_csv(path_data[k] + "_Pago" + csv)

df

,CVE_INEGI,CVE_ENT,CVE_MUN,Usuarios_1*,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Tarifa,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_1*,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1.0,1.0,254974.0,-102.295872,21.811400,27.182119,13.074257,20.159075,522.051880,-482.033417,507.881513,-8.212147,1.347620,442.348606,845118.000000,1.374286e+11,23.691382,4.142857,7.245778,162160.723559,NaN,11.211926,116635.924759,1,1.920337e+08,1.693105e+08,1643.929187,3.613441e+08,1417.180334,4926.241999,3282.312812,8.369044e+08
1,1002,1.0,2.0,11557.0,-102.045590,22.126479,26.460970,12.328507,19.395846,429.399714,-494.662554,556.378653,-12.173154,0.947158,358.376980,46075.142857,3.491166e+09,23.034981,4.142857,0.847945,75705.155679,NaN,11.211926,54337.426603,1,7.759527e+06,6.846582e+06,1466.045333,1.460611e+07,1263.832183,4393.190501,2927.145168,3.382902e+07
2,1003,1.0,3.0,16736.0,-102.704911,21.900640,27.009948,12.518081,19.773620,621.113330,-448.972848,462.771497,-4.712476,0.947242,519.512183,55283.142857,4.358511e+09,23.082358,4.142857,0.598719,78733.145869,NaN,11.211926,92335.783229,1,1.122328e+07,9.884636e+06,1463.024548,2.110791e+07,1261.228059,4384.138338,2921.113789,4.888776e+07
3,1004,1.0,4.0,3788.0,-102.297038,22.360619,26.527926,12.449918,19.528949,448.636825,-512.037785,557.611886,-13.815264,0.775050,377.125221,15363.000000,1.449504e+09,23.041425,4.142857,1.196649,94221.940180,NaN,11.211926,12838.354454,1,2.510776e+06,2.213835e+06,1446.818812,4.724612e+06,1247.257597,4335.575798,2888.756986,1.094261e+07
4,1005,1.0,5.0,30597.0,-102.445700,21.932112,26.925495,12.886637,19.890029,531.063800,-460.285867,479.792441,-7.156598,1.232661,442.897088,112079.000000,1.318374e+10,23.204577,4.142857,2.242050,117305.832917,NaN,11.211926,49989.509158,1,2.141233e+07,1.878628e+07,1524.017942,4.019860e+07,1313.808571,4566.912768,3042.894825,9.310345e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,3923.0,-101.714324,22.407345,26.185634,12.639281,19.411589,357.411519,-506.235879,590.748675,-22.991064,1.840217,288.391072,18889.000000,1.104084e+09,23.055471,4.000000,0.508554,58432.484498,NaN,11.268627,37142.598662,1,2.097387e+06,1.593920e+06,1091.490174,3.691307e+06,940.939806,3220.896221,2129.406047,8.353660e+06
2465,32055,32.0,55.0,8847.0,-102.851571,22.298130,26.407706,12.344591,19.528930,558.532327,-469.145342,489.120322,-9.259408,0.741970,468.364202,29902.000000,2.396876e+09,22.681947,4.142857,0.138660,80134.667679,NaN,11.268627,215650.560221,1,5.127218e+06,3.901293e+06,1183.799394,9.028511e+06,1020.516719,3493.293009,2309.493615,2.043209e+07
2466,32056,32.0,56.0,37702.0,-102.678055,22.735730,25.725354,11.868202,18.824115,439.723002,-530.293745,536.387068,-19.077788,1.257958,374.477437,142958.571429,2.484715e+10,22.429208,4.142857,3.272704,173750.774044,NaN,11.268627,43682.101050,1,2.610999e+07,1.966034e+07,1408.242957,4.577032e+07,1214.002549,4155.607192,2747.364235,1.035811e+08
2467,32057,32.0,57.0,15674.0,-102.312009,22.755625,26.292193,12.041775,19.151676,404.788083,-550.953892,588.182495,-21.708939,0.905130,344.888589,18421.428571,1.414295e+09,22.864307,4.000000,0.843291,76845.878314,NaN,11.268627,21844.700672,1,1.143491e+07,9.139649e+06,1522.680135,2.057456e+07,1312.655289,4493.301734,2970.621599,4.656152e+07
